# Pipe for create embeddings

## Imports

In [2]:
import torch
from transformers import AutoTokenizer, AutoModel
import functions

import pandas as pd
import numpy as np
import seaborn as sns

import sklearn.metrics as metrics

## Model

In [3]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base")

## Data

In [5]:
df = pd.read_json("CLR10.jsonl", lines=True)
big_boys = df.copy()

In [6]:
big_boys["completion"] = big_boys["completion"].str.len()
big_boys = big_boys[big_boys.completion.astype(int) > 1000]
big_boys

,task_id,completion
204,HumanEval/20,1297
218,HumanEval/21,1027
245,HumanEval/24,1154
247,HumanEval/24,1319
262,HumanEval/26,1896
503,HumanEval/50,1260
545,HumanEval/54,1280
657,HumanEval/65,1224
673,HumanEval/67,1024
678,HumanEval/67,1389


In [7]:
sample = df.loc[204]["completion"]
sample1 = df.loc[1]["completion"]
print(len(sample))

1297


In [18]:
sample_emb = functions.get_embedding(model, tokenizer, sample1)
sample1_emb = functions.get_embedding(model, tokenizer, sample1)

In [12]:
sample_emb.size()

torch.Size([499, 768])

In [13]:
sample1_emb.size()

torch.Size([124, 768])

In [19]:
room = [sample_emb, sample1_emb]

In [20]:
kernel = functions.kernel_entropy(room)
kernel

ValueError: Found array with dim 3. check_pairwise_arrays expected <= 2.

In [4]:
functions.deaths(model, tokenizer)

HumanEval/0
HumanEval/0
HumanEval/0
HumanEval/0
HumanEval/0
HumanEval/0
HumanEval/0
HumanEval/0
HumanEval/0
HumanEval/0
HumanEval/1
HumanEval/1
HumanEval/1
HumanEval/1
HumanEval/1
HumanEval/1
HumanEval/1
HumanEval/1
HumanEval/1
HumanEval/1
HumanEval/2
HumanEval/2
HumanEval/2
HumanEval/2
HumanEval/2
HumanEval/2
HumanEval/2
HumanEval/2
HumanEval/2
HumanEval/2
HumanEval/3
HumanEval/3
HumanEval/3
HumanEval/3
HumanEval/3
HumanEval/3
HumanEval/3
HumanEval/3
HumanEval/3
HumanEval/3
HumanEval/4
HumanEval/4
HumanEval/4
HumanEval/4
HumanEval/4
HumanEval/4
HumanEval/4
HumanEval/4
HumanEval/4
HumanEval/4
HumanEval/5
HumanEval/5
HumanEval/5
HumanEval/5
HumanEval/5
HumanEval/5
HumanEval/5
HumanEval/5
HumanEval/5
HumanEval/5
HumanEval/6
HumanEval/6
HumanEval/6
HumanEval/6
HumanEval/6
HumanEval/6
HumanEval/6
HumanEval/6
HumanEval/6
HumanEval/6
HumanEval/7
HumanEval/7
HumanEval/7
HumanEval/7
HumanEval/7
HumanEval/7
HumanEval/7
HumanEval/7
HumanEval/7
HumanEval/7
HumanEval/8
HumanEval/8
HumanEval/8
Huma

Token indices sequence length is longer than the specified maximum sequence length for this model (551 > 512). Running this sequence through the model will result in indexing errors


HumanEval/49
HumanEval/49
HumanEval/50
HumanEval/50
HumanEval/50
HumanEval/50
HumanEval/50
HumanEval/50
HumanEval/50
HumanEval/50
HumanEval/50
HumanEval/50
HumanEval/51
HumanEval/51
HumanEval/51
HumanEval/51
HumanEval/51
HumanEval/51
HumanEval/51
HumanEval/51
HumanEval/51
HumanEval/51
HumanEval/52
HumanEval/52
HumanEval/52
HumanEval/52
HumanEval/52
HumanEval/52
HumanEval/52
HumanEval/52
HumanEval/52
HumanEval/52
HumanEval/53
HumanEval/53
HumanEval/53
HumanEval/53
HumanEval/53
HumanEval/53
HumanEval/53
HumanEval/53
HumanEval/53
HumanEval/53
HumanEval/54
HumanEval/54
HumanEval/54
HumanEval/54
HumanEval/54
HumanEval/54
HumanEval/54
HumanEval/54
HumanEval/54
HumanEval/54
HumanEval/55
HumanEval/55
HumanEval/55
HumanEval/55
HumanEval/55
HumanEval/55
HumanEval/55
HumanEval/55
HumanEval/55
HumanEval/55
HumanEval/56
HumanEval/56
HumanEval/56
HumanEval/56
HumanEval/56
HumanEval/56
HumanEval/56
HumanEval/56
HumanEval/56
HumanEval/56
HumanEval/57
HumanEval/57
HumanEval/57
HumanEval/57
HumanEval/57

## Pipe

In [9]:
# emb = functions.get_embedding(model, tokenizer, torch, "def max(a,b): if a>b: return a else return b")

In [10]:
# emb